In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data as utils
import getData

In [7]:
senLen = 59
indexSize = 15144
embeddingSize = 300
kernelSizes = [3,4,5]
numFilters = 3
embeddings = None
XY = None
model = None
lossFn = nn.CrossEntropyLoss().type(torch.FloatTensor)
opt = None

In [20]:
def makeEmbeddings(numWords,index,X,senWords,embed):
    inputData = torch.zeros([len(X),1,senWords,embeddingSize],dtype=torch.float64)
    for i,sen in enumerate(X):
        senIndex = torch.zeros([1,senWords,embeddingSize],dtype=torch.float64)
        for j,word in enumerate(sen):
            senIndex[0,j]= embed[j-1]
        inputData[i][0] = Variable(torch.DoubleTensor(senIndex[0]))
    return inputData

def makeModel(lr):
    model = nn.Sequential(
            nn.Conv2d(1,200,kernel_size=(3,embeddingSize),stride=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(200),
            nn.Dropout(p=0.2),
            nn.Conv2d(200,300,(4,1),stride=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(300),
            nn.Dropout(p=0.2),
            nn.Conv2d(300,500,(5,1),stride=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(500),
            nn.Dropout(p=0.2),
            nn.MaxPool2d((50,1)),
            Flatten(), 
            nn.Linear(500,2),
    )
    opt = optim.Adam(model.parameters(),lr=lr)
    return model,opt

def train(numEpochs=1):
    print("Model",model)
    bestAcc = 0
    for epoch in range(numEpochs):
        print("Epoch #",epoch)
        total_loss = 0
        model.train()
        for i,(x,y) in enumerate(XY):
            xVar = Variable(x).float()
            yVar = Variable(y).type(torch.LongTensor)
            scores = model(xVar)
            loss = lossFn(scores,yVar)
            if i == 0: 
                print("i = %d, loss = %.4f" % (i + 1, loss.data[0]))
            if (i + 1) % 100 == 0:
                print("i = %d, loss = %.4f " % (i + 1, loss.data[0]))
            opt.zero_grad()
            loss.backward()
            total_loss += loss.data[0] 
            opt.step()
        currAcc = checkAccuracy(XY)
        if currAcc > bestAcc: 
            bestAcc = currAcc
            saveCheckpoint(model,'model.pt')
        
        print("total_loss %.4f"% total_loss/float(i))
def checkAccuracy(dataSet):
    #if(self.XY.dataset.train):
    #    print('Checking accuracy on validation set')
    #else:
    #    print('Checking accuracy on test set')
    numCorrect = 0
    numSamples = 0
    model.eval()
    for i,(x,y) in enumerate(dataSet):
        xVar = Variable(x).float()
        yVar = Variable(y).type(torch.LongTensor)
        scores = model(xVar)
        _,preds = scores.data.cpu().max(1)
        numCorrect += (preds == yVar).sum()
        numSamples += preds.size(0)
    acc = float(numCorrect)/numSamples
    print('Got %d / %d correct (%.2f)' % (numCorrect, numSamples, 100 * acc))

class Flatten(nn.Module):
    def forward(self,x):
        N,C,H,W = x.size()
        return x.view(N,-1)

In [13]:
print('calling getData...')
X,index,XY,Y,max_sent_len = getData.runTrain()

calling getData...
15144 51
true
59
True


In [10]:
Xte,XYte,Yte,max_sent_ = getData.getTest(index,max_sent_len)

In [14]:
print('making embeddings...')
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

making embeddings...


In [15]:
embeddings = torch.randn(len(index)+1,300)
for k,v in index.items():
    try: 
        embeddings[v-1].copy_(torch.from_numpy(word_vectors.get_vector(k)))
    except KeyError:
        continue 
embeddings[0] = 0

In [16]:
inputData = makeEmbeddings(len(index),index,X,senLen,embeddings)
#inputDataTe = makeEmbeddings(len(index),index,Xte,senLen,embeddings)

In [17]:
print('making model...')
model,opt=makeModel(lr=1e-5)
print(type(inputData), type(Y))
my_datasetTr = utils.TensorDataset(inputData, Y)
#my_datasetTe = utils.TensorDataset(inputDataTe, Yte)
my_dataloaderTr = utils.DataLoader(my_datasetTr,batch_size=64)
#my_dataloaderTe = utils.DataLoader(my_datasetTe,batch_size=64)
XY = my_dataloaderTr

making model...
<class 'torch.Tensor'> <class 'torch.Tensor'>


In [21]:
print('calling train...')
train(50)

calling train...
Model Sequential(
  (0): Conv2d(1, 200, kernel_size=(3, 300), stride=(1, 1))
  (1): ReLU(inplace)
  (2): BatchNorm2d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.2)
  (4): Conv2d(200, 300, kernel_size=(4, 1), stride=(1, 1))
  (5): ReLU(inplace)
  (6): BatchNorm2d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.2)
  (8): Conv2d(300, 500, kernel_size=(5, 1), stride=(1, 1))
  (9): ReLU(inplace)
  (10): BatchNorm2d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): Dropout(p=0.2)
  (12): MaxPool2d(kernel_size=(50, 1), stride=(50, 1), padding=0, dilation=1, ceil_mode=False)
  (13): Linear(in_features=500, out_features=2, bias=True)
)
Epoch # 0


RuntimeError: size mismatch, m1: [32000 x 1], m2: [500 x 2] at /Users/soumith/code/builder/wheel/pytorch-src/aten/src/TH/generic/THTensorMath.c:2033

In [18]:
def save_checkpoint(state, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
save_checkpoint(model,  'model.pt')

In [51]:
print('checking training accuracy...')
checkAccuracy(XY)

checking training accuracy...
Got 42259 / 76961 correct (54.91)


In [22]:
train(10)

Epoch # 0


/Users/user1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


i = 100, loss = 5.3657
i = 200, loss = 4.3702
i = 300, loss = 5.7993
i = 400, loss = 4.5596
i = 500, loss = 4.6797
i = 600, loss = 4.4845
i = 700, loss = 5.8012
i = 800, loss = 5.0226
i = 900, loss = 5.4300
i = 1000, loss = 6.1454
i = 1100, loss = 4.9737
i = 1200, loss = 3.4508
Epoch # 1
i = 100, loss = 5.5240
i = 200, loss = 4.0358
i = 300, loss = 6.1639
i = 400, loss = 4.8161
i = 500, loss = 4.9376
i = 600, loss = 4.1924
i = 700, loss = 5.5555
i = 800, loss = 4.1912
i = 900, loss = 5.6428
i = 1000, loss = 6.3821
i = 1100, loss = 5.2004
i = 1200, loss = 4.2636
Epoch # 2
i = 100, loss = 5.3878
i = 200, loss = 4.0733
i = 300, loss = 5.5333
i = 400, loss = 4.7414
i = 500, loss = 5.1319
i = 600, loss = 4.0356
i = 700, loss = 6.0306
i = 800, loss = 4.4343
i = 900, loss = 5.7235
i = 1000, loss = 5.7273
i = 1100, loss = 4.9162
i = 1200, loss = 4.2799
Epoch # 3
i = 100, loss = 4.6574
i = 200, loss = 4.4554
i = 300, loss = 5.9660
i = 400, loss = 4.5642
i = 500, loss = 5.6273
i = 600, loss = 5.

In [23]:
checkAccuracy(XY)

Got 34702 / 76961 correct (45.09)


In [15]:
print('checking validation accuracy...')
checkAccuracy(my_dataloaderTe)

checking validation accuracy...
Got 912 / 1821 correct (50.08)


In [14]:
train(5)

Epoch # 0


/Users/user1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


i = 100, loss = 9.7361
i = 200, loss = 9.6903
i = 300, loss = 9.7456
i = 400, loss = 9.6905
i = 500, loss = 9.6392
i = 600, loss = 9.6411
i = 700, loss = 9.6473
i = 800, loss = 9.6686
i = 900, loss = 9.6868
i = 1000, loss = 9.7353
i = 1100, loss = 9.6816
i = 1200, loss = 9.5365
Epoch # 1
i = 100, loss = 9.6092
i = 200, loss = 9.5382
i = 300, loss = 9.6364
i = 400, loss = 9.6277
i = 500, loss = 9.5093
i = 600, loss = 9.5198
i = 700, loss = 9.6371
i = 800, loss = 9.4575
i = 900, loss = 9.5999
i = 1000, loss = 9.6197
i = 1100, loss = 9.5490
i = 1200, loss = 9.4070
Epoch # 2
i = 100, loss = 9.5561
i = 200, loss = 9.3913
i = 300, loss = 9.5359
i = 400, loss = 9.4710
i = 500, loss = 9.4327
i = 600, loss = 9.3857
i = 700, loss = 9.4832
i = 800, loss = 9.3978
i = 900, loss = 9.5204
i = 1000, loss = 9.5192
i = 1100, loss = 9.4498
i = 1200, loss = 9.2692
Epoch # 3
i = 100, loss = 9.3439
i = 200, loss = 9.2677
i = 300, loss = 9.4322
i = 400, loss = 9.3178
i = 500, loss = 9.2859
i = 600, loss = 9.